# Importing required libraries, reading training and test data
Добавляем необходимые библиотеки, считываем данные для обучения и проверки нашей модели

In [ ]:
import pandas as pd    
# список стоп-слов
from nltk.corpus import stopwords 
# удаление тегов
from bs4 import BeautifulSoup
# работа с регулярными выражениями
import re
# импорт тренировочной и тестовой выборок
train = pd.read_csv("../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip", header=0, \
                    delimiter="\t", quoting=3)
test = pd.read_csv("../input/word2vec-nlp-tutorial/testData.tsv.zip", header=0, \
                    delimiter="\t", quoting=3)

# Describing function for converting a raw review to a string of words
Функция преобразует "сырой" обзор на фильм, поступающий в виде строки, в строку значимых слов из этого обзора

In [ ]:
def review_to_words( raw_review ):
    # удаляем элементы HTML из текста
    review_text = BeautifulSoup(raw_review).get_text() 
    # убираем все кроме букв   
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    # оставляем строчные буквы, разбиваем на отдельные слова
    words = letters_only.lower().split()                             
    # конвертируем список в множество для ускорения поиска и удаления слов
    stops = set(stopwords.words("english"))                  
    # убираем слова, которые не несут смысловой нагрузки в предложении
    meaningful_words = [w for w in words if not w in stops]   
    # получаем строку со словами, разделенными пробелом
    return( " ".join( meaningful_words ))   

# Cleaning and parsing the training set movie reviews
С помощью описанной выше функции очищаем выборку (убираем теги, оставляем только буквы, разбиваем выборку на слова, преобразуем список в множество для ускорения поиска и удаления слов, которые не несут смысловой нагрузки в предложении)

In [ ]:
num_reviews = train["review"].size
clean_train_reviews = []
for i in range( 0, num_reviews ):
    # вывод сообщения о ходе обработки
    if( (i+1)%1000 == 0 ):
        print("Proceed %d / %d\n" % ( i+1, num_reviews ))                                                                  
    clean_train_reviews.append( review_to_words( train["review"][i] ))

# Creating the bag of words
Создаем словарный запас и вектора признаков из полученных тренировочных данных с помощью функции fit_transform()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 25000) 
train_data_features = vectorizer.fit_transform(clean_train_reviews)
train_data_features = train_data_features.toarray()

# Training model with RandomForestClassifier
Для тренировки модели используется алгоритм Random Forest, для начала инициализируем модель, берем 120 деревьев

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators = 120) 
forest = forest.fit( train_data_features, train["sentiment"] )

# Getting results
Создаем пустой список и добавляем в него очищенные обзоры, после чего получаем результаты работы на тестовой выборке

In [ ]:
clean_test_reviews = []
for i in range(0,len(test["review"])):
    clean_review = review_to_words(test["review"][i])
    clean_test_reviews.append(clean_review)

test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

result = forest.predict(test_data_features)

# Writing results into a .csv file
Записываем результаты обработки в выходной файл

In [ ]:
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

output.to_csv( "submission.csv", index=False, quoting=3 )